In [77]:
import gurobipy as gp
import math

In [78]:
m = gp.Model("Curtain")

In [79]:
LENGTHS = [
    352,
    350,
    276,
    200,
    0,
]

In [80]:
TRACK_LENGTH = 140

In [81]:
def calculate_max_track_count(lengths, track_length):
    count = 0

    for l in lengths:
        count += math.ceil(l / track_length)

    return count

In [82]:
MAX_TRACKS = calculate_max_track_count(LENGTHS, TRACK_LENGTH)
MAX_TRACKS

10

In [83]:
print(
    f"We have {len(LENGTHS)-1} windows need to install curtains, the track's length is {TRACK_LENGTH}.\n" \
    f"And the windows' lengths are {','.join([f'{i}' for i in LENGTHS[:-1]])}. So how can we minimize the tracks' usage.\n" \
    f"We also know that we can cutting and combine tracks, but we want to minimize the cutting numbers as well as the usage of tracks."
)


We have 4 windows need to install curtains, the track's length is 140.
And the windows' lengths are 352,350,276,200. So how can we minimize the tracks' usage.
We also know that we can cutting and combine tracks, but we want to minimize the cutting numbers as well as the usage of tracks.


In [84]:
x = m.addVars(MAX_TRACKS, len(LENGTHS), vtype=gp.GRB.INTEGER, name="x")
z = m.addVars(MAX_TRACKS, vtype=gp.GRB.BINARY, name="z")

m.update()

In [85]:
# Constraint 0: All values greater than 0
for l in range(len(LENGTHS)):
    for t in range(MAX_TRACKS):
        m.addConstr(x[t, l] >= 0, "greater_than_0")

In [86]:
# Constraint 1: Curtain should have enough length
for i, l in enumerate(LENGTHS[:-1]):
    m.addConstr(sum(x[t, i] * z[t] for t in range(MAX_TRACKS)) == l, name="curtain_should_have_enough_length")

In [87]:
# Constraint 2: Track should be full length

for t in range(MAX_TRACKS):
    m.addConstr(sum(x[t, l] for l in range(len(LENGTHS))) == TRACK_LENGTH, name="track_should_be_full_length")

In [88]:
# Objectives
m.setObjectiveN(sum(z[t] for t in range(MAX_TRACKS)), index=0)
m.setObjectiveN(sum(x[t, l] for t in range(MAX_TRACKS) for l in range(len(LENGTHS[:-1]))), index=1)

In [89]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 60 columns and 100 nonzeros
Model fingerprint: 0x9dde4786
Model has 4 quadratic constraints
Variable types: 0 continuous, 60 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
  QRHS range       [2e+02, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined)...
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
--------------------------------------------------------

In [90]:
for t in range(MAX_TRACKS):
    for l in range(len(LENGTHS)):
        if x[t, l].X > 0:
            print(f"Track {t} for Curtain {l}, with length: {x[t, l].X}")

Track 0 for Curtain 2, with length: 80.0
Track 0 for Curtain 3, with length: 60.0
Track 1 for Curtain 2, with length: 140.0
Track 2 for Curtain 1, with length: 84.0
Track 2 for Curtain 2, with length: 56.0
Track 3 for Curtain 1, with length: 140.0
Track 4 for Curtain 0, with length: 14.0
Track 4 for Curtain 1, with length: 126.0
Track 5 for Curtain 0, with length: 140.0
Track 6 for Curtain 0, with length: 140.0
Track 7 for Curtain 0, with length: 58.0
Track 7 for Curtain 4, with length: 82.0
Track 8 for Curtain 4, with length: 140.0
Track 9 for Curtain 3, with length: 140.0


In [91]:
for l in range(len(LENGTHS)):
    length = LENGTHS[l]
    print(f"Curtain {l} should have length: {length}")

    sum_length = 0
    for t in range(MAX_TRACKS):
        if x[t, l].X > 0:
            sum_length += x[t, l].X
            print(f"track {t}: {x[t, l].X}", end=" ")
    print(f" => {sum_length}", end="")

    print("")
    print("-" * 32)

Curtain 0 should have length: 352
track 4: 14.0 track 5: 140.0 track 6: 140.0 track 7: 58.0  => 352.0
--------------------------------
Curtain 1 should have length: 350
track 2: 84.0 track 3: 140.0 track 4: 126.0  => 350.0
--------------------------------
Curtain 2 should have length: 276
track 0: 80.0 track 1: 140.0 track 2: 56.0  => 276.0
--------------------------------
Curtain 3 should have length: 200
track 0: 60.0 track 9: 140.0  => 200.0
--------------------------------
Curtain 4 should have length: 0
track 7: 82.0 track 8: 140.0  => 222.0
--------------------------------
